In [ ]:
pip install fedml

In [ ]:
################

In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
from PIL import Image
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import ast
from flsim.interfaces.model_manager import ModelManager
from flsim.privacy.common import PrivacySetting
from flsim.utils.config_utils import DictConfigWrapper
from flsim.trainers.sync_trainer import SyncTrainer
from flsim.data.data_loader import FLDataLoader
from flsim.utils.simple_model import SimpleConvNet  # Just a dummy fallback
from flsim.utils.example_models import ModelTrainer
from flsim.channels.message import Message
from flsim.optimizers.async_optimizers import FedAvgWithLRDecay
import hydra
import logging

# ==== Setup ====
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
train_images_dir = os.path.join(base_path, "Training Images")
data_file = os.path.join(base_path, "data.xlsx")
csv_dir = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/"

# ==== Preprocess ====
df = pd.read_excel(data_file)
label_cols = ["N","D","G","C","A","H","M","O"]
rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].astype(int).tolist()})
df_images = pd.DataFrame(rows)
df_images["labels"] = df_images["labels"].apply(ast.literal_eval)

# ==== Dataset ====
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row["filename"])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(image), label

# ==== Split clients ====
NUM_CLIENTS = 5
splits = random_split(df_images, [len(df_images)//NUM_CLIENTS]*NUM_CLIENTS)
clients_data = [OcularDataset(s.dataset.iloc[s.indices], train_images_dir) for s in splits]

# ==== Model ====
def get_model():
    model = EfficientNet.from_pretrained("efficientnet-b0")
    model._fc = nn.Linear(model._fc.in_features, 8)
    return model

# ==== Trainer ====
class CustomTrainer(ModelTrainer):
    def __init__(self, model, optimizer, criterion, device, client_id):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        self.client_id = client_id

    def train(self, train_loader):
        self.model.train()
        for epoch in range(3):
            for x, y in train_loader:
                x, y = x.to(self.device), y.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(x)
                loss = self.criterion(output, y)
                loss.backward()
                self.optimizer.step()

    def evaluate(self, val_loader, round_num):
        self.model.eval()
        y_true, y_pred = [], []
        total_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(self.device), y.to(self.device)
                output = self.model(x)
                loss = self.criterion(output, y)
                total_loss += loss.item()
                preds = (torch.sigmoid(output) > 0.3).int().cpu().numpy()
                y_true.extend(y.cpu().numpy())
                y_pred.extend(preds)

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average='samples', zero_division=0)
        rec = recall_score(y_true, y_pred, average='samples', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='samples', zero_division=0)

        # Save CSV
        csv_path = os.path.join(csv_dir, f"DFL_Client_{self.client_id}.csv")
        log = pd.DataFrame([[round_num, total_loss, acc, prec, rec, f1]],
                           columns=["round", "loss", "accuracy", "precision", "recall", "f1"])
        if os.path.exists(csv_path):
            log.to_csv(csv_path, mode="a", header=False, index=False)
        else:
            log.to_csv(csv_path, index=False)

        print(f"Client {self.client_id} - Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")
        return acc

# ==== Training Loop ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.0]*8).to(device))
NUM_ROUNDS = 20

# Simulate Mesh DFL
logs = [pd.DataFrame(columns=["round", "loss", "accuracy", "precision", "recall", "f1"]) for _ in range(NUM_CLIENTS)]
clients = []

for i in range(NUM_CLIENTS):
    model = get_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    ds = clients_data[i]
    train_len = int(0.8 * len(ds))
    train_ds, val_ds = random_split(ds, [train_len, len(ds)-train_len])
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=8)
    trainer = CustomTrainer(model, optimizer, criterion, device, i+1)
    clients.append({"trainer": trainer, "val_loader": val_loader, "train_loader": train_loader})

# Mesh update simulation: each client sends params to next neighbor
for round_num in range(1, NUM_ROUNDS + 1):
    print(f"\n=== Round {round_num} ===")
    # Train
    for c in clients:
        c["trainer"].train(c["train_loader"])
    # Mesh-style param sharing
    new_weights = [c["trainer"].model.state_dict() for c in clients]
    for i in range(NUM_CLIENTS):
        next_i = (i + 1) % NUM_CLIENTS
        clients[next_i]["trainer"].model.load_state_dict(new_weights[i])
    # Eval
    for c in clients:
        c["trainer"].evaluate(c["val_loader"], round_num)


ModuleNotFoundError: No module named 'flsim.interfaces.model_manager'

In [2]:
# ==============================
# Improved Mesh-based EfficientNet DFL Code
# With Faster Convergence, Cosine Scheduler, Lower Threshold, and Per-Round CSV Saving
# ==============================

import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
from PIL import Image
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# ======== Dataset Preparation ==========
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform if transform else T.Compose([
            T.Resize((224, 224)),
            T.RandomHorizontalFlip(),
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            T.RandomRotation(10),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# ======== Model Definition ==============
def get_model():
    model = EfficientNet.from_pretrained("efficientnet-b2")
    model._fc = nn.Linear(model._fc.in_features, 8)
    return model

# ======== Load & Prepare Data ===========
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
train_images_dir = os.path.join(base_path, "Training Images")
data_file = os.path.join(base_path, "data.xlsx")
df = pd.read_excel(data_file)
label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]

rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})

df_images = pd.DataFrame(rows)

# ======== Data Split ============
total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# ======== Config ================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 5
ROUNDS = 30
lr = 5e-4

# ======== Weighted Aggregation ============
def aggregate_weights(weights_list, client_sizes):
    total = sum(client_sizes)
    avg_weights = []
    for weights in zip(*weights_list):
        avg = sum(w * s for w, s in zip(weights, client_sizes)) / total
        avg_weights.append(avg)
    return avg_weights

# ======== Mesh Topology ==========
neighbors = {i: [j for j in range(5) if j != i] for i in range(5)}

# ======== Initialize Clients ============
clients = []
for idx in range(5):
    df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
    dataset = OcularDataset(df, train_images_dir)
    split = int(0.8 * len(dataset))
    train_ds, val_ds = random_split(dataset, [split, len(dataset) - split])
    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=16)
    clients.append({
        "model": get_model().to(device),
        "train_loader": train_loader,
        "val_loader": val_loader
    })

# ======== Training Loop ============
logs = [pd.DataFrame(columns=['Round', 'Loss', 'Accuracy', 'Precision', 'Recall', 'F1']) for _ in range(5)]
criterion = nn.BCEWithLogitsLoss()

log_dir = "mesh_logs_EfficientNet"
os.makedirs(log_dir, exist_ok=True)

for rnd in range(1, ROUNDS + 1):
    print(f"\n=== Round {rnd} ===")
    weights_list, client_sizes = [], []

    for idx, client in enumerate(clients):
        model = client['model']
        model.train()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

        for _ in range(EPOCHS):
            for images, labels in client['train_loader']:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                loss = criterion(model(images), labels)
                loss.backward()
                optimizer.step()
            scheduler.step()

        # Evaluation
        model.eval()
        y_true, y_pred = [], []
        total_loss = 0
        with torch.no_grad():
            for images, labels in client['val_loader']:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                total_loss += criterion(outputs, labels).item()
                preds = torch.sigmoid(outputs).cpu().numpy()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend((preds > 0.3).astype(int))  # Lower threshold

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)

        print(f"Client {idx+1} - Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")

        logs[idx] = pd.concat([
            logs[idx],
            pd.DataFrame([[rnd, total_loss, acc, prec, rec, f1]], columns=logs[idx].columns)
        ], ignore_index=True)

        weights_list.append([val.cpu().detach().numpy() for val in model.state_dict().values()])
        client_sizes.append(len(client['train_loader'].dataset))

    # Aggregation
    for i in range(5):
        neighbor_indices = neighbors[i] + [i]
        selected_weights = [weights_list[j] for j in neighbor_indices]
        selected_sizes = [client_sizes[j] for j in neighbor_indices]
        new_weights = aggregate_weights(selected_weights, selected_sizes)
        state_dict = clients[i]['model'].state_dict()
        new_state = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), new_weights)}
        clients[i]['model'].load_state_dict(new_state)

    # Save logs after each round
    for i in range(5):
        logs[i].to_csv(
            f"C:/Users/aitoo/OneDrive/Desktop/FYP-Code/mesh_logs_EfficientNet/client_{i+1}_performance_log.csv",
            index=False
        )


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to C:\Users\aitoo/.cache\torch\hub\checkpoints\efficientnet-b2-8bb594d6.pth


100%|██████████| 35.1M/35.1M [01:44<00:00, 351kB/s] 


Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2

=== Round 1 ===
Client 1 - Acc: 0.2929 | Prec: 0.4961 | Rec: 0.6339 | F1: 0.5342


C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\939386771.py:147: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[idx] = pd.concat([


Client 2 - Acc: 0.2964 | Prec: 0.5190 | Rec: 0.6512 | F1: 0.5548


...

C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\939386771.py:147: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[idx] = pd.concat([


Client 5 - Acc: 0.2643 | Prec: 0.4595 | Rec: 0.6065 | F1: 0.5035


C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\939386771.py:147: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[idx] = pd.concat([



=== Round 2 ===
Client 1 - Acc: 0.3036 | Prec: 0.5119 | Rec: 0.6173 | F1: 0.5371
Client 2 - Acc: 0.3179 | Prec: 0.5155 | Rec: 0.6101 | F1: 0.5369
Client 3 - Acc: 0.2857 | Prec: 0.5327 | Rec: 0.6601 | F1: 0.5614
Client 4 - Acc: 0.2536 | Prec: 0.4628 | Rec: 0.6095 | F1: 0.5069
Client 5 - Acc: 0.3500 | Prec: 0.5095 | Rec: 0.5851 | F1: 0.5269

=== Round 3 ===
Client 1 - Acc: 0.3786 | Prec: 0.5530 | Rec: 0.6494 | F1: 0.5780
Client 2 - Acc: 0.3679 | Prec: 0.5446 | Rec: 0.6238 | F1: 0.5619
Client 3 - Acc: 0.2964 | Prec: 0.4917 | Rec: 0.5690 | F1: 0.5060
Client 4 - Acc: 0.2893 | Prec: 0.4643 | Rec: 0.5518 | F1: 0.4842
Client 5 - Acc: 0.3286 | Prec: 0.5250 | Rec: 0.6387 | F1: 0.5545

=== Round 4 ===
Client 1 - Acc: 0.3714 | Prec: 0.5381 | Rec: 0.5917 | F1: 0.5439
Client 2 - Acc: 0.3429 | Prec: 0.5268 | Rec: 0.6095 | F1: 0.5438
Client 3 - Acc: 0.3393 | Prec: 0.5494 | Rec: 0.6274 | F1: 0.5614
Client 4 - Acc: 0.2607 | Prec: 0.4685 | Rec: 0.5756 | F1: 0.4954
Client 5 - Acc: 0.3500 | Prec: 0.5435 |

KeyboardInterrupt: 

EfficientNet

In [1]:
# ==============================
# Improved Mesh-based EfficientNet DFL Code
# With Weighted Aggregation, Augmentations, Scheduler, Better Loss, and Per-Round CSV Saving
# ==============================

import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
from PIL import Image
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# ======== Dataset Preparation ==========
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform if transform else T.Compose([
            T.Resize((224, 224)),
            T.RandomHorizontalFlip(),
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            T.RandomRotation(10),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# ======== Model Definition ==============
def get_model():
    model = EfficientNet.from_pretrained("efficientnet-b0")
    model._fc = nn.Linear(model._fc.in_features, 8)
    return model

# ======== Load & Prepare Data ===========
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
train_images_dir = os.path.join(base_path, "Training Images")
data_file = os.path.join(base_path, "data.xlsx")
df = pd.read_excel(data_file)
label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]

rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})

df_images = pd.DataFrame(rows)

# Calculate pos_weight for BCEWithLogitsLoss
label_matrix = np.array(df_images["labels"].to_list())
counts = label_matrix.sum(axis=0)
total = len(df_images)
pos_weight = torch.tensor((total - counts) / counts, dtype=torch.float32)

# ======== Data Split ============
total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# ======== Config ================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 10
ROUNDS = 50
lr = 1e-4

# ======== Weighted Aggregation ============
def aggregate_weights(weights_list, client_sizes):
    total = sum(client_sizes)
    avg_weights = []
    for weights in zip(*weights_list):
        avg = sum(w * s for w, s in zip(weights, client_sizes)) / total
        avg_weights.append(avg)
    return avg_weights

# ======== Mesh Topology ==========
neighbors = {i: [j for j in range(5) if j != i] for i in range(5)}

# ======== Initialize Clients ============
clients = []
for idx in range(5):
    df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
    dataset = OcularDataset(df, train_images_dir)
    split = int(0.8 * len(dataset))
    train_ds, val_ds = random_split(dataset, [split, len(dataset) - split])
    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=16)
    clients.append({
        "model": get_model().to(device),
        "train_loader": train_loader,
        "val_loader": val_loader
    })

# ======== Training Loop ============
logs = [pd.DataFrame(columns=['Round', 'Loss', 'Accuracy', 'Precision', 'Recall', 'F1']) for _ in range(5)]
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))

# Make sure output directory exists
log_dir = "mesh_logs_EfficientNet"
os.makedirs(log_dir, exist_ok=True)

for rnd in range(1, ROUNDS + 1):
    print(f"\n=== Round {rnd} ===")
    weights_list, client_sizes = [], []

    for idx, client in enumerate(clients):
        model = client['model']
        model.train()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

        for _ in range(EPOCHS):
            for images, labels in client['train_loader']:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                loss = criterion(model(images), labels)
                loss.backward()
                optimizer.step()
            scheduler.step()

        # Evaluation
        model.eval()
        y_true, y_pred = [], []
        total_loss = 0
        with torch.no_grad():
            for images, labels in client['val_loader']:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                total_loss += criterion(outputs, labels).item()
                preds = torch.sigmoid(outputs).cpu().numpy()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend((preds > 0.5).astype(int))

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)

        print(f"Client {idx+1} - Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")

        logs[idx] = pd.concat([
            logs[idx],
            pd.DataFrame([[rnd, total_loss, acc, prec, rec, f1]], columns=logs[idx].columns)
        ], ignore_index=True)

        weights_list.append([val.cpu().detach().numpy() for val in model.state_dict().values()])
        client_sizes.append(len(client['train_loader'].dataset))

    # Aggregation
    for i in range(5):
        neighbor_indices = neighbors[i] + [i]
        selected_weights = [weights_list[j] for j in neighbor_indices]
        selected_sizes = [client_sizes[j] for j in neighbor_indices]
        new_weights = aggregate_weights(selected_weights, selected_sizes)
        state_dict = clients[i]['model'].state_dict()
        new_state = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), new_weights)}
        clients[i]['model'].load_state_dict(new_state)

    # ======== Save Logs After Each Round ==========
    for i in range(5):
        logs[i].to_csv(
        f"C:/Users/aitoo/OneDrive/Desktop/FYP-Code/mesh_logs_EfficientNet/client_{i+1}_performance_log.csv",
        index=False
        )


Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0

=== Round 1 ===
Client 1 - Acc: 0.1464 | Prec: 0.3869 | Rec: 0.6339 | F1: 0.4563


C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\427025815.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[idx] = pd.concat([


Client 2 - Acc: 0.1607 | Prec: 0.4115 | Rec: 0.6458 | F1: 0.4798


C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\427025815.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[idx] = pd.concat([


Client 3 - Acc: 0.1286 | Prec: 0.3740 | Rec: 0.6274 | F1: 0.4478


...

C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\427025815.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[idx] = pd.concat([


Client 5 - Acc: 0.1679 | Prec: 0.4265 | Rec: 0.6524 | F1: 0.4892


C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\427025815.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[idx] = pd.concat([



=== Round 2 ===
Client 1 - Acc: 0.1679 | Prec: 0.3991 | Rec: 0.6429 | F1: 0.4677
Client 2 - Acc: 0.2107 | Prec: 0.4554 | Rec: 0.6667 | F1: 0.5185
Client 3 - Acc: 0.1393 | Prec: 0.3928 | Rec: 0.6476 | F1: 0.4676
Client 4 - Acc: 0.1714 | Prec: 0.3929 | Rec: 0.6149 | F1: 0.4580
Client 5 - Acc: 0.2321 | Prec: 0.4732 | Rec: 0.6792 | F1: 0.5292

=== Round 3 ===
Client 1 - Acc: 0.2286 | Prec: 0.4562 | Rec: 0.6661 | F1: 0.5170
Client 2 - Acc: 0.1786 | Prec: 0.4377 | Rec: 0.6667 | F1: 0.5032
Client 3 - Acc: 0.2107 | Prec: 0.4280 | Rec: 0.6161 | F1: 0.4839
Client 4 - Acc: 0.1643 | Prec: 0.4027 | Rec: 0.6131 | F1: 0.4633
Client 5 - Acc: 0.2357 | Prec: 0.4795 | Rec: 0.6845 | F1: 0.5362

=== Round 4 ===
Client 1 - Acc: 0.2036 | Prec: 0.4277 | Rec: 0.6036 | F1: 0.4770
Client 2 - Acc: 0.2000 | Prec: 0.4539 | Rec: 0.6500 | F1: 0.5067
Client 3 - Acc: 0.2071 | Prec: 0.4255 | Rec: 0.6131 | F1: 0.4808
Client 4 - Acc: 0.2071 | Prec: 0.4235 | Rec: 0.6226 | F1: 0.4811
Client 5 - Acc: 0.2393 | Prec: 0.4530 |

KeyboardInterrupt: 

Swin Transformer

In [3]:
# === Swin Transformer with DFL-based Mesh Topology (FedAvg) ===
# ==============================================================
# Uses simple average instead of weighted aggregation
# ==============================================================
import os
import copy
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import ast
import timm

# === Dataset & Loader ===
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform if transform else T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# === Model ===
def get_model():
    model = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=8)
    return model

# === Settings ===
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
data_file = os.path.join(base_path, "data.xlsx")
image_dir = os.path.join(base_path, "Training Images")

label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]
df = pd.read_excel(data_file)
rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
df_images = pd.DataFrame(rows)
df_images["labels"] = df_images["labels"].apply(lambda x: ast.literal_eval(str(x)))

total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# === Mesh Topology Neighbors ===
neighbors = {
    0: [1, 2, 3, 4],
    1: [0, 2, 3, 4],
    2: [0, 1, 3, 4],
    3: [0, 1, 2, 4],
    4: [0, 1, 2, 3]
}

# === Device ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Logs ===
os.makedirs("mesh_logs_swin_fedavg", exist_ok=True)
logs = [pd.DataFrame(columns=["Round", "Loss", "Accuracy", "Precision", "Recall", "F1"])
        for _ in range(5)]

# === Training ===
clients = []
for idx in range(5):
    dataset = OcularDataset(clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices], image_dir)
    split = int(0.8 * len(dataset))
    train_ds, val_ds = random_split(dataset, [split, len(dataset) - split])
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=8)
    model = get_model().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    pos_weight = torch.tensor([3.0] * 8).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    clients.append({
        'model': model, 'optimizer': optimizer, 'train_loader': train_loader,
        'val_loader': val_loader, 'criterion': criterion
    })

# === Mesh Rounds ===
for round_num in range(1, 31):
    print(f"\n=== Round {round_num} ===")
    weights = []

    # Local Training
    for i, client in enumerate(clients):
        client['model'].train()
        for epoch in range(3):
            for x, y in client['train_loader']:
                x, y = x.to(device), y.to(device)
                client['optimizer'].zero_grad()
                pred = client['model'](x)
                loss = client['criterion'](pred, y)
                loss.backward()
                client['optimizer'].step()

    # Evaluation
    for i, client in enumerate(clients):
        client['model'].eval()
        y_true, y_pred = [], []
        val_loss = 0
        with torch.no_grad():
            for x, y in client['val_loader']:
                x, y = x.to(device), y.to(device)
                out = client['model'](x)
                val_loss += client['criterion'](out, y).item()
                preds = torch.sigmoid(out).cpu().numpy()
                y_true.extend(y.cpu().numpy())
                y_pred.extend((preds > 0.3).astype(int))

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)

        logs[i] = pd.concat([logs[i], pd.DataFrame([[round_num, val_loss, acc, prec, rec, f1]],
                                                   columns=logs[i].columns)], ignore_index=True)
        weights.append(copy.deepcopy(client['model'].state_dict()))

        print(f"Client {i+1} - Loss: {val_loss:.4f}, Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")

    # FedAvg Aggregation
    for i in range(5):
        neighbor_weights = [weights[j] for j in neighbors[i]] + [weights[i]]
        avg_weights = copy.deepcopy(neighbor_weights[0])
        for key in avg_weights.keys():
            for j in range(1, len(neighbor_weights)):
                avg_weights[key] += neighbor_weights[j][key]
            avg_weights[key] /= len(neighbor_weights)
        clients[i]['model'].load_state_dict(avg_weights)

# === Save Logs ===
for i, log in enumerate(logs):
    log.to_csv(f"C:/Users/aitoo/OneDrive/Desktop/FYP-Code/mesh_logs_EfficientNet/client_{i+1}_performance_log.csv", index=False)

print("\n✅ Mesh-based DFL with Swin Transformer (FedAvg) Completed.")


c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



=== Round 1 ===


C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\655558932.py:133: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[i] = pd.concat([logs[i], pd.DataFrame([[round_num, val_loss, acc, prec, rec, f1]],


Client 1 - Loss: 20.4446, Acc: 0.0179, Prec: 0.3485, Rec: 0.8833, F1: 0.4865


C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\655558932.py:133: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[i] = pd.concat([logs[i], pd.DataFrame([[round_num, val_loss, acc, prec, rec, f1]],


...

C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\655558932.py:133: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[i] = pd.concat([logs[i], pd.DataFrame([[round_num, val_loss, acc, prec, rec, f1]],


Client 4 - Loss: 20.9530, Acc: 0.0179, Prec: 0.3474, Rec: 0.8732, F1: 0.4832


C:\Users\aitoo\AppData\Local\Temp\ipykernel_21552\655558932.py:133: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  logs[i] = pd.concat([logs[i], pd.DataFrame([[round_num, val_loss, acc, prec, rec, f1]],


Client 5 - Loss: 20.9755, Acc: 0.0000, Prec: 0.3283, Rec: 0.8792, F1: 0.4684

=== Round 2 ===


KeyboardInterrupt: 

mobilenet_v3_large

In [ ]:
# mobilenet_v3_large - Mesh-based Decentralized Federated Learning (DFL)
# ===========================================================
# Uses FedAvg for model aggregation
# ===========================================================

import os
import copy
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T, models
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import ast

# === Dataset & Loader ===
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform if transform else T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# === Model ===
def get_model():
    model = models.mobilenet_v3_large(pretrained=True)
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, 8)
    return model

# === FedAvg Aggregation ===
def fedavg(state_dicts):
    avg_state = copy.deepcopy(state_dicts[0])
    for key in avg_state.keys():
        for i in range(1, len(state_dicts)):
            avg_state[key] += state_dicts[i][key]
        avg_state[key] /= len(state_dicts)
    return avg_state

# === Settings ===
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
data_file = os.path.join(base_path, "data.xlsx")
image_dir = os.path.join(base_path, "Training Images")

label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]
df = pd.read_excel(data_file)
rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
df_images = pd.DataFrame(rows)
df_images["labels"] = df_images["labels"].apply(lambda x: ast.literal_eval(str(x)))

total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# === Mesh Topology Neighbors ===
neighbors = {
    0: [1, 2, 3, 4],
    1: [0, 2, 3, 4],
    2: [0, 1, 3, 4],
    3: [0, 1, 2, 4],
    4: [0, 1, 2, 3]
}

# === Device ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Logs ===
os.makedirs("mesh_logs_mobilenetv3", exist_ok=True)
logs = [pd.DataFrame(columns=["Round", "Loss", "Accuracy", "Precision", "Recall", "F1"])
        for _ in range(5)]

# === Training ===
clients = []
for idx in range(5):
    dataset = OcularDataset(clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices], image_dir)
    split = int(0.8 * len(dataset))
    train_ds, val_ds = random_split(dataset, [split, len(dataset) - split])
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=8)
    model = get_model().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    pos_weight = torch.tensor([3.0] * 8).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    clients.append({
        'model': model, 'optimizer': optimizer, 'train_loader': train_loader,
        'val_loader': val_loader, 'criterion': criterion
    })

# === Mesh Rounds ===
for round_num in range(1, 31):
    print(f"\n=== Round {round_num} ===")
    weights = []

    # Local Training
    for i, client in enumerate(clients):
        client['model'].train()
        for epoch in range(3):
            for x, y in client['train_loader']:
                x, y = x.to(device), y.to(device)
                client['optimizer'].zero_grad()
                pred = client['model'](x)
                loss = client['criterion'](pred, y)
                loss.backward()
                client['optimizer'].step()

    # Evaluation
    for i, client in enumerate(clients):
        client['model'].eval()
        y_true, y_pred = [], []
        val_loss = 0
        with torch.no_grad():
            for x, y in client['val_loader']:
                x, y = x.to(device), y.to(device)
                out = client['model'](x)
                val_loss += client['criterion'](out, y).item()
                preds = torch.sigmoid(out).cpu().numpy()
                y_true.extend(y.cpu().numpy())
                y_pred.extend((preds > 0.3).astype(int))

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)

        logs[i] = pd.concat([logs[i], pd.DataFrame([[round_num, val_loss, acc, prec, rec, f1]],
                                                   columns=logs[i].columns)], ignore_index=True)
        weights.append(copy.deepcopy(client['model'].state_dict()))

        print(f"Client {i+1} - Loss: {val_loss:.4f}, Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")

    # Aggregation using FedAvg
    for i in range(5):
        neighbor_weights = [weights[j] for j in neighbors[i]] + [weights[i]]
        agg_weights = fedavg(neighbor_weights)
        clients[i]['model'].load_state_dict(agg_weights)

# === Save Logs ===
for i, log in enumerate(logs):
    log.to_csv(f"mesh_logs_mobilenetv3/client_{i+1}_performance_log.csv", index=False)

print("\n✅ Mesh-based DFL with MobileNetV3 (FedAvg) Completed.")
